# VGG-16 on CIFAR-10
This notebook is used to experiment with VGG-16 on CIFAR-10 dataset.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import sys

sys.path.append('../../')
sys.path.append('../../src/')

import src.general as general
import src.dataset_models as data
import src.metrics as metrics
import src.evaluation as eval
import src.plot as plot
import src.compression.distillation as distill
import src.compression.pruning as prune


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Get model
vgg_weights = tv.models.VGG16_Weights.DEFAULT
vgg16 = tv.models.vgg16(weights=vgg_weights)
# Modify the last layer to have 10 output classes (CIFAR-10 has 10 classes)
vgg16.classifier[6] = nn.Linear(4096, 10)


In [4]:
# Load the model from a checkpoint
checkpoint = torch.load("/workspace/volume/models/vgg16_cifar10.pt")
vgg16.load_state_dict(checkpoint)

<All keys matched successfully>

In [5]:
# Get dataset
dataset = data.supported_datasets["CIFAR-10"]

In [6]:
# Get transforms
vgg_cifar10_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Set transforms
dataset.set_transforms(vgg_cifar10_transform)

In [7]:
general.get_device()

device(type='cuda')

In [8]:
before_results = eval.get_results(vgg16, dataset)

Test: 100%|██████████| 157/157 [00:26<00:00,  5.99it/s]


Test loss: 0.2082
Test score: 93.9689
Could not calculate FLOPS


In [9]:
plot.print_results(**before_results)

==================================== RESULTS ===================================
Loss: 0.208209
Score: 93.968949
Time per data point: 10.4277 ms
Model Size: 512.33 MB
Number of parameters: 134301514
Number of MACs: 15499459200


In [10]:
ignored_layers = prune.get_layers_not_to_prune(vgg16.features)
print(ignored_layers)

[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]


In [25]:
example_inputs = general.get_example_inputs(dataset.train_loader)
input_batch = next(iter(dataset.train_loader))

In [12]:
vgg16()

[tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           ...,
           [-2.1179, -2.1179, -0.2513,  ..., -0.5767, -0.5767, -0.5767],
           [-2.1179, -2.1179, -0.2342,  ..., -0.5938, -0.5938, -0.5938],
           [-2.1179, -2.1179, -0.2342,  ..., -0.5938, -0.5938, -0.5938]],
 
          [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
           [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
           [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
           ...,
           [-2.0357, -2.0357,  0.1527,  ..., -0.2325, -0.2325, -0.2325],
           [-2.0357, -2.0357,  0.1527,  ..., -0.2500, -0.2500, -0.2500],
           [-2.0357, -2.0357,  0.1527,  ..., -0.2500, -0.2500, -0.2500]],
 
          [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
           [-

In [13]:
mnist = data.supported_datasets["MNIST"]

In [24]:
general.get_example_inputs(mnist.train_loader).shape

torch.Size([64, 1, 28, 28])

In [15]:
len(general.get_example_input_batch(dataset.train_loader))

2

In [22]:
pruned_model = prune.magnitude_pruning_structured(vgg16, dataset, 0.5, fineTune=False, iterative_steps=5, layers=vgg16.features)

TypeError: conv2d() received an invalid combination of arguments - got (list, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)


In [ ]:
after_results = eval.get_results(pruned_model, dataset)

Test: 100%|██████████| 157/157 [00:25<00:00,  6.04it/s]


Test loss: 0.2122
Test score: 93.6306
Could not calculate FLOPS


In [ ]:
plot.print_before_after_results(before_results, after_results)

============================= RESULTS BEFORE & AFTER ===========================
Loss: 0.207255 -> 0.212175 (2.37%)
Score: 93.859475 -> 93.630573 (-0.24%)
Time per data point: 10.3823 ms -> 10.3398 ms (-0.41%)
Model Size: 512.33 MB -> 512.33 MB (-0.00%)
Number of parameters: 134301514 -> 134301514 (-0.00%)
Number of MACs: 15499459200 -> 15499459200 (-0.00%)


In [ ]:
torch.save(vgg16.state_dict(), "/workspace/volume/models/vgg16_cifar10_pruned_50.pt")